In [ ]:
import household_contact_tracing.branching_process_models as bpm
from household_contact_tracing.simulation_controller import SimulationController

# Notebook purpose
To provide minimal working examples of various model functionalities.

# Individual-level contact tracing
Previously, we implemented household-level contact tracing, where upon detection of a case in a household, all members of the household have their contacts traced. Household-level contact tracing is motivated by the significant levels of pre-symptomatic and asymptomatic transmission observed in SARS-CoV-2, which means that by the time you detect a case in a household there is non-zero probability that other members of the household have already propagated the infection. Household-level contact tracing is a more pro-active approach to stopping infection, however often difficult to implement in practice.

Individual-level contact tracing is different, in that an individual only has their contacts traced when they are detected as a case. This will immediately initiate quarantine of their household members, but those household members will not propagate contact tracing until they are detected as an infection. At a high level, this is how contact tracing is implemented by NHS England Test & Trace.

Note: I've not implemented the infection dynamics examples here, since it ought to be exactly the same code as in household_level_contact_tracing.ipynb.

# 0.1  - Default parameters
We initialise some default parameters that we will modify as we go

In [ ]:
# some default parameters that we use in nearly all simulations without modifying
# specifies a fairly typical epidemic and contact tracing process

base_params = {
    'outside_household_infectivity_scaling': 0.7,
    'contact_tracing_success_prob': 0.7,
    'overdispersion': 0.32,
    'asymptomatic_prob': 0.2,
    'asymptomatic_relative_infectivity': 0.35,
    'infection_reporting_prob': 0.5,
    'contact_trace': True,
    'test_delay': 2,
    'contact_trace_delay': 1,
    'incubation_period_delay': 5,
    'symptom_reporting_delay': 1,
    'household_pairwise_survival_prob': 0.2,
    'reduce_contacts_by': 0.3,
    'starting_infections': 1,
    'node_will_uptake_isolation_prob': 1,
    'self_isolation_duration': 7,
    'quarantine_duration': 10,
    'transmission_probability_multiplier': 1,
    'propensity_imperfect_quarantine': 0,
    'global_contact_reduction_imperfect_quarantine': 0
}

Now, we add some parameters that are specific to the individual-level contact tracing model.

Previously, all contact were being traced, regardless of when the contact occurred
we now explicitely only trace contact that occurred 2 days prior to symptom onset, and 5 days after symptom onset, in line with NHS England Test & Trace policy.

In [ ]:
base_params['number_of_days_to_trace_backwards'] = 2
base_params['number_of_days_to_trace_forwards'] = 5

A model of recall decay is now incorporated, where the probability of recalling contacts decays as the amount of time that has passed since the contact has occurred increases.

In [ ]:
# this is badly parameterised, but a value of 1 implies there is no recall decay
base_params['recall_probability_fall_off'] = 1

An optional contact tracing behaviour is included, where if a traced individual develops symptoms, then this can be used to propagate contact tracing without the need for a test. In other words, symptom onset in a traced individual is sufficient evidence to initiate tracing, due to the high proabbility that they are infected with SARS-CoV-2, and not a similar influenza-like illness.

In [ ]:
# Default value is true, in line with current policy
base_params['probable_infections_need_test'] = True

# 1.1 - Simple individual-level contact tracing

In [ ]:
base_params

In [ ]:
controller = SimulationController(bpm.IndividualLevelContactTracing(base_params))
controller.run_simulation(15)

In [ ]:
controller.graph_pyvis_view.open_in_browser = True
controller.graph_view.set_display(False)
controller.run_simulation(20)